# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on April 13 (with 2-hour grace period) and worth 25 points

### Overview

This milestone builds on your work from Milestone 1. You will:

1. Evaluate baseline models using default settings.
2. Engineer new features and re-evaluate models.
3. Use feature selection techniques to find promising subsets.
4. Select the top 3 models and fine-tune them for optimal performance.

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone. 


In [29]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



## Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling.

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:** 

- You will not use the testing set during this milestone — it’s reserved for final evaluation later.
- You will have to redo the scaling step when you introduce new features (which have to be scaled as well).


In [55]:
df = pd.read_csv("zillow_cleaned.csv")
X = df.drop(columns=["taxvaluedollarcnt"])
y = df["taxvaluedollarcnt"]

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=random_state)

In [57]:
scaler = StandardScaler()   
X_train_scaled = scaler.fit_transform(X_train)

### Part 1: Baseline Modeling [3 pts]

Apply the following regression models to the scaled training dataset using **default parameters**:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each model:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV RMSE Score** across all folds in a table. 


In [20]:
# Repeated cross-validation with default 5 folds and 10 repeats
repeated_cv = RepeatedKFold(random_state=random_state)

models = [LinearRegression(), 
          Ridge(random_state=random_state), 
          Lasso(random_state=random_state), 
          DecisionTreeRegressor(random_state=random_state), 
          BaggingRegressor(random_state=random_state), 
          RandomForestRegressor(random_state=random_state), 
          GradientBoostingRegressor(random_state=random_state)]

baseline_means = []
baseline_std = []

start = time.time()

for model in models:
    cv_rmse_scores = np.sqrt(-cross_val_score(model, X_train_scaled, y_train, cv=repeated_cv, scoring="neg_mean_squared_error", n_jobs=4))
    mean_cv_rmse = np.mean(cv_rmse_scores)
    std_cv = np.std(cv_rmse_scores)
    
    baseline_means.append(dollar_format(mean_cv_rmse, 0))
    baseline_std.append(dollar_format(std_cv, 0))

end = time.time()

baseline_df = pd.DataFrame({
    "Model": ["Linear Regression", "Ridge", "Lasso", "Decision Tree", "Bagging Regressor", "Random Forest", "Gradient Boosting"],
    "Mean RMSE": baseline_means,
    "Std RMSE": baseline_std
})

format_hms(end - start)

'00:07:32'

In [21]:
baseline_df

,Model,Mean RMSE,Std RMSE
0,Linear Regression,"$319,442","$4,195"
1,Ridge,"$319,442","$4,195"
2,Lasso,"$319,442","$4,195"
3,Decision Tree,"$389,703","$5,343"
4,Bagging Regressor,"$294,504","$3,965"
5,Random Forest,"$282,642","$3,397"
6,Gradient Boosting,"$287,794","$3,682"


### Part 1: Discussion [2 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which models perform best overall?
  - Which are most stable (lowest std)?
  - Any signs of overfitting or underfitting?

> Your text here

### Part 2: Feature Engineering [3 pts]

Consider **at least three new features** based on your Milestone 1, Part 5. Examples include:
- Polynomial terms
- Log or interaction terms
- Groupings or transformations of categorical features

Add these features to `X_train` and then:
- Scale using `StandardScaler` 
- Re-run all models listed above (using default settings again).
- Report updated RMSE scores (mean and std) across repeated CV in a table. 

**Note:**  Recall that this will require creating a new version of the dataset, so effectively you may be running "polynomial regression" using `LinearRegression`. 

In [58]:
# Add polynomial terms
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(X_train[["bathroomcnt", "bedroomcnt"]])
poly_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(["bathroomcnt", "bedroomcnt"]))
X_train = X_train.reset_index(drop=True)
X_train = pd.concat([X_train.drop(columns=["bathroomcnt", "bedroomcnt"]), poly_df], axis=1)

# Add log terms
X_train["log_finishedsquarefeet12"] = np.log(X_train["finishedsquarefeet12"])
X_train["log_calculatedfinishedsquarefeet"] = np.log(X_train["calculatedfinishedsquarefeet"])

# Scale using StandardScaler
X_train_scaled = scaler.fit_transform(X_train)

In [60]:
added_features_means = []
added_features_std = []

start = time.time()

for model in models:
    cv_rmse_scores = np.sqrt(-cross_val_score(model, X_train_scaled, y_train, cv=repeated_cv, scoring="neg_mean_squared_error", n_jobs=4))
    mean_cv_rmse = np.mean(cv_rmse_scores)
    std_cv = np.std(cv_rmse_scores)
    
    added_features_means.append(dollar_format(mean_cv_rmse, 0))
    added_features_std.append(dollar_format(std_cv, 0))

end = time.time()

added_features_df = pd.DataFrame({
    "Model": ["Linear Regression", "Ridge", "Lasso", "Decision Tree", "Bagging Regressor", "Random Forest", "Gradient Boosting"],
    "Mean RMSE": added_features_means,
    "Std RMSE": added_features_std
})

format_hms(end - start)

/home/jason8924/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.831e+12, tolerance: 8.876e+11
  model = cd_fast.enet_coordinate_descent(
/home/jason8924/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.309e+12, tolerance: 8.896e+11
  model = cd_fast.enet_coordinate_descent(
/home/jason8924/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

'00:10:05'

In [61]:
added_features_df

,Model,Mean RMSE,Std RMSE
0,Linear Regression,"$317,432","$4,134"
1,Ridge,"$317,424","$4,130"
2,Lasso,"$317,415","$4,126"
3,Decision Tree,"$390,760","$5,342"
4,Bagging Regressor,"$294,990","$3,945"
5,Random Forest,"$282,932","$3,471"
6,Gradient Boosting,"$287,796","$3,655"


### Part 2: Discussion [2 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?

- Were there any unexpected results?



> Your text here

### Part 3: Feature Selection [3 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features.
- Report updated RMSE scores (mean and std) across repeated CV in a table.


In [83]:
# Forward Feature Selection

def forward_feature_selection(X, y, model, 
                              scoring='neg_mean_squared_error', 
                              cv=5, 
                              tol=None,               # None = no delta cutoff
                                                      # use 0.0 for "no further improvements"
                                                      # and 1e-4 for "point of diminishing returns"                                      
                              max_features=None,      # None = use all features
                              n_jobs=-1,
                              verbose=False
                             ):
    selected_features = []                            # List to store the order of features selected
    remaining_features = list(X.columns)              # Features not yet selected
    best_scores = []                                  # List to store the CV score after each feature addition
    previous_score = float('inf')                     # Initialize previous score for improvement comparison

    # Track the best subset of features and its corresponding score
    
    best_feature_set = None                           # Best combination of features found so far
    best_score = float('inf')                         # Best CV score observed so far
    
    while remaining_features:
        scores = {}                                   # Dictionary to hold CV scores for each candidate feature
        for feature in remaining_features:
            current_features = selected_features + [feature]
            
            # Compute the CV score for the current set of features (negated MSE, so lower is better)
            cv_score = -cross_val_score(model, X[current_features], y, 
                                        scoring=scoring, cv=cv, n_jobs=n_jobs
                                       ).mean()
            scores[feature] = np.sqrt(cv_score)

        # Select the feature that minimizes the CV score
        best_feature = min(scores, key=scores.get)
        current_score = scores[best_feature]
            
        # Check if the improvement is significant based on the tolerance (tol)
        if tol is not None and previous_score - current_score < tol:
            if verbose:
                print("Stopping early due to minimal improvement.")
            break

        # Add the best feature to the selected list and update score trackers
        selected_features.append(best_feature)
        best_scores.append(current_score)
        remaining_features.remove(best_feature)
        previous_score = current_score

        if verbose:
            print(f"\nFeatures: {selected_features[-3:]}, CV Score (MSE): {current_score:.4f}")
        
        # Update the best subset if the current score is better than the best so far
        if current_score < best_score:
            best_score = current_score
            best_feature_set = selected_features.copy()
        
        # Check if the maximum number of features has been reached
        if max_features is not None and len(selected_features) >= max_features:
            break

    return (
        selected_features,      # List of features in the order they were selected (this will be ALL features if max_features == None
        best_scores,            # List of cross-validation scores corresponding to each addition in the previous list
        best_feature_set,       # The subset of features that achieved the best CV score.
        best_score              # The best CV score
    )

In [84]:
best_feature_sets = []
best_cv_rmse_scores = []

start = time.time()

for model in tqdm(models, desc="Performing Feature Selection"):
    selected_features, best_scores, best_feature_set, best_score = forward_feature_selection(pd.DataFrame(X_train_scaled, columns=X_train.columns), y_train, model, n_jobs=3)
    best_feature_sets.append(best_feature_set)
    best_cv_rmse_scores.append(dollar_format(best_score,0))

feature_selection_df = pd.DataFrame({
    "Model": ["Linear Regression", "Ridge", "Lasso", "Decision Tree", "Bagging Regressor", "Random Forest", "Gradient Boosting"],
    "Best Feature Set": best_feature_sets,
    "Best CV RMSE Score": best_cv_rmse_scores
})

end = time.time()
format_hms(end - start)

Performing Feature Selection:  29%|███████████████▋                                       | 2/7 [01:09<02:53, 34.65s/it]/home/jason8924/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.826e+13, tolerance: 8.975e+11
  model = cd_fast.enet_coordinate_descent(
/home/jason8924/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.185e+12, tolerance: 8.830e+11
  model = cd_fast.enet_coordinate_descent(
/home/jason8924/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might

'02:06:25'

In [85]:
feature_selection_df

,Model,Best Feature Set,Best CV RMSE Score
0,Linear Regression,"[finishedsquarefeet12, regionidzip, buildingqu...","$316,353"
1,Ridge,"[finishedsquarefeet12, regionidzip, buildingqu...","$316,353"
2,Lasso,"[finishedsquarefeet12, regionidzip, buildingqu...","$316,353"
3,Decision Tree,"[regionidzip, calculatedbathnbr, propertylandu...","$306,528"
4,Bagging Regressor,"[regionidzip, bathroomcnt^2, propertylandusety...","$294,144"
5,Random Forest,"[regionidzip, bathroomcnt, propertylandusetype...","$282,498"
6,Gradient Boosting,"[finishedsquarefeet12, regionidzip, yearbuilt,...","$287,773"


In [88]:
feature_selection_means = []
feature_selection_stds = []

start = time.time()

for model, best_feature_set in zip(models, best_feature_sets):
    cv_rmse_scores = np.sqrt(-cross_val_score(model, pd.DataFrame(X_train_scaled, columns=X_train.columns)[best_feature_set], y_train, cv=repeated_cv, scoring="neg_mean_squared_error", n_jobs=3))
    mean_cv_rmse = np.mean(cv_rmse_scores)
    std_cv = np.std(cv_rmse_scores)
    
    feature_selection_means.append(dollar_format(mean_cv_rmse, 0))
    feature_selection_stds.append(dollar_format(std_cv, 0))
    
end = time.time()

feature_selection_df = pd.DataFrame({
    "Model": ["Linear Regression", "Ridge", "Lasso", "Decision Tree", "Bagging Regressor", "Random Forest", "Gradient Boosting"],
    "Mean RMSE": feature_selection_means,
    "Std RMSE": feature_selection_stds
})

format_hms(end - start)

/home/jason8924/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.718e+14, tolerance: 8.896e+11
  model = cd_fast.enet_coordinate_descent(
/home/jason8924/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.918e+14, tolerance: 8.876e+11
  model = cd_fast.enet_coordinate_descent(
/home/jason8924/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

'00:10:51'

In [89]:
feature_selection_df

,Model,Mean RMSE,Std RMSE
0,Linear Regression,"$316,311","$3,971"
1,Ridge,"$316,311","$3,971"
2,Lasso,"$316,311","$3,971"
3,Decision Tree,"$307,285","$4,155"
4,Bagging Regressor,"$294,777","$3,932"
5,Random Forest,"$282,817","$3,259"
6,Gradient Boosting,"$287,735","$3,718"


### Part 3: Discussion [2 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?

- How did feature selection differ between linear and tree-based models?

> Your text here

### Part 4: Fine-Tuning Your Top 3 Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far.

1. Choose the top 3 models based on performance and interpretability from earlier parts.
2. For each model:
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, or other techniques from previous homeworks. 
   - Experiment with different versions of your feature engineering and preprocessing — treat these as additional tunable components.
3. Report the mean and standard deviation of CV RMSE score for each model in a summary table.



In [5]:
# Add as many code cells as you need

### Part 4: Discussion [4 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?
- Provide a ranking of your three models and explain your reasoning — not just based on RMSE, but also interpretability, training time, or generalizability.
- Conclude by considering whether this workflow has produced the results you expected. Typically, you would repeat steps 2 - 4 and also reconsider the choices you made in Milestone 1 when cleaning the dataset, until reaching the point of diminishing returns; do you think that would that have helped here?

> Your text here